In [28]:
MODEL = "intfloat/multilingual-e5-small" 
INDEX_NAME = "llm-doc"
import sys
sys.path.append('..')

In [35]:
import os 
from elasticsearch import Elasticsearch

#https://discuss.elastic.co/t/issue-connecting-python-to-elasticsearch-in-docker-environment/361507/2
es_client = Elasticsearch(
    hosts=["https://localhost:9200"],
    basic_auth=('elastic', os.getenv("ELASTIC_PASSWORD")),
    verify_certs=False,
    max_retries=30,
    retry_on_timeout=True,
    request_timeout=30,
)

In [30]:
import json

with open('../data/documents-llm.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

docs = docs_raw[0]['documents']

In [31]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(MODEL)
for doc in docs:
    # Transforming the title into an embedding using the model
    doc["question_encoded"] = model.encode(doc["question"]).tolist()

In [33]:
index_settings = {
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "question_encoded": {"type": "dense_vector", "dims": len(docs[0]['question_encoded']), "index": True, "similarity": "cosine"},
        }
    }
}

In [37]:
es_client.indices.delete(index=INDEX_NAME, ignore_unavailable=True)
es_client.indices.create(index=INDEX_NAME, body=index_settings)
for doc in docs: 
    es_client.index(index="llm-doc", document=doc)

/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usa

CONTINUE ON THIS NOTEBOOK
https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/demo_es.ipynb

https://www.elastic.co/docs/solutions/search/search-approaches

Full-text search uses bm25 on the text and query directly. 
Vector search can be based ond dense vectors (and knn to find similar vectors) or sparse vectors. 
Among sparse vector models, ES provides ELSER. This basically uses BERT encoding to generate synonyms based on sequences of words in the query and content. 
The n highest scoring words are taken as expansion tokens and mapped back to a sparse vector.

Hybrid search uses vector and full-text search together.
Semantic search provides managed workflows that use vector search under the hood.

https://www.elastic.co/docs/reference/elasticsearch/clients/python/examples
https://github.com/elastic/elasticsearch-labs/tree/main/notebooks/search

ELSER is most easy to use, but not available in the free version of ES.
Plan
- Use hugging face to create a dense representation of question field of documents
- Create mapping (https://www.elastic.co/docs/solutions/search/vector/knn)
- Create index 
